# ❄️ Build an End-to-End Distributed Machine Learning Workflow in Snowflake ❄️

# Best Practice: Integrate your git repositories and use a container runtime python environment. 
On the right hand side, you'll notice that the contents of the git repository are available for use. You can select a branch, pull the latest files, and, on the upper right hand corner of the screen, you can make commits and push updates back to your repository. 


In this Notebook ([on Container Runtime](https://docs.snowflake.com/developer-guide/snowflake-ml/notebooks-on-spcs)), we will develop a machine learning model that accurately predicts the "mortgage response" (e.g., loan approval, offer acceptance) based on borrower characteristics and loan details.

We will showcase all the typical steps in a machine learning pipeline using native capabilities in Snowflake through this use case:

### 1. `FEATURE ENGINEERING:` Use [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/overview) to track engineered features
- Store feature defintions in a feature store for reproducible computation of ML features
      
### 2. `DISTRIBUTED MODEL TRAINING, DEPLOYMENT, & INFERENCE`
- Distributed XGBoost via [Snowflake's Distributed Modeling Classes](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/modeling_distributors) - multi-node, multi-GPU
- Explore [Snowflake Model Registry](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/overview) capabilities such as **metadata tracking, model metrics tracking, and inference/deployment**
- [Deploy model to Snowpark Container Services](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/container)

### 3. `ML OBSERVABILITY:` Set up a [Model Monitor](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/model-observability) to track 1 year of predicted and actual loan repayments
- Compute performance metrics such a F1, Precision, Recall
- Inspect model drift (i.e. how much has the average predicted repayment rate changed day-to-day)
- Compare models side-by-side to understand which model should be used in production
Identify and understand data issues

# Best practice: Augment our robust container runtimes with pip package management to tailor a python environment to your needs

In [1]:
# Built-in
import math
import pickle
from datetime import datetime

# Third-party
import pandas as pd
import numpy as np
import shap
import sklearn
import streamlit as st
from xgboost import XGBClassifier

# Snowflake ML
from snowflake.ml.registry import Registry
from snowflake.ml.feature_store import FeatureStore, FeatureView, Entity, CreationMode
from snowflake.ml.runtime_cluster import scale_cluster, get_nodes

# Snowpark Core
from snowflake.snowpark import DataFrame, Window
from snowflake.snowpark.context import get_active_session
from snowflake.snowpark.functions import (
    col,
    to_timestamp,
    min,
    max,
    month,
    dayofmonth,
    dayofweek,
    dayofyear,
    avg,
    date_add,
    sql_expr,
)
from snowflake.snowpark.types import IntegerType

# Initialize session
session = get_active_session()

# Use schema DEFAULT_SCHEMA to create new Snowflake objects in this project
session.use_schema('DEFAULT_SCHEMA')
session.use_role('E2E_ML_ROLE')

session

import logging
#logging.basicConfig(level=logging.WARN)
logging.getLogger().setLevel(logging.WARN)

In [ ]:
# Let's set a version (must be a string) that will be used through all the artifacts 
# that will be created for our project.
VERSION_NUM = 'ML303_V1'

We can read in the .csv file and store the data as a Snowflake table. Then, we can read in the data as a [Snowpark dataframe](https://docs.snowflake.com/en/developer-guide/snowpark/python/working-with-dataframes).

In [ ]:
try:
    print("Reading table data...")
    df = session.table("MORTGAGE_LENDING_DEMO_DATA")
    df.show(5)
except:
    print("Table not found! Uploading data to snowflake table")
    df_pandas = pd.read_csv("MORTGAGE_LENDING_DEMO_DATA.csv.zip")
    session.write_pandas(df_pandas, "MORTGAGE_LENDING_DEMO_DATA", auto_create_table=True)
    df = session.table("MORTGAGE_LENDING_DEMO_DATA")
    df.show(5)

In [ ]:
# Update timestamp info using Snowpark APIs
current_time = datetime.now()
df_max_time = datetime.strptime(str(df.select(max("TS")).collect()[0][0]), "%Y-%m-%d %H:%M:%S.%f")
    
timedelta = current_time- df_max_time
df = df.with_columns(
        ["TIMESTAMP", "MONTH", "DAY_OF_YEAR", "DOTW"],
        [date_add(to_timestamp("TS"), timedelta.days-1),
            month("TIMESTAMP"),
            dayofyear("TIMESTAMP"),
            dayofweek("TIMESTAMP")])

# Feature Engineering with Snowpark APIs

We will create a number of features within `create_mortgage_features()`:
- Timestamp features (i.e. month, day of year, day of week)
- Income and Loan features
- County-level income stats
- High income flag
- Time-based rolling average.


# Best Practice: Use the language you are most comfortable with to load and transform your Snowflake data, efficiently

The features below are created using a combination of the Snowpark Dataframe API and SQL expressions. 
You can find the Snowpark API reference documentation [here](https://docs.snowflake.com/en/developer-guide/snowpark/reference/python/1.2.0/index). Popular native python libraries like Pandas are always available for use within a Snowflake Notebook, and the Snowpark Pandas API can also be used to scale Pandas-based operational syntax to be applied to Snowflake table data.  

In [ ]:
def create_mortgage_features(df):
    # Step 2: Income and loan features (Per-row features)
    df = df.with_columns(
        ["LOAN_AMOUNT", "LOAN_PURPOSE_NAME", "INCOME", "INCOME_LOAN_RATIO"],
        [
            col("LOAN_AMOUNT_000s")*1000,
            col("LOAN_PURPOSE_NAME"),
            col("APPLICANT_INCOME_000s")*1000,
            col("INCOME")/col("LOAN_AMOUNT")
        ]
    )
    
    # Step 3: County-level income stats (Per-group features)
    county_income_df = df.group_by(["COUNTY_NAME"]).agg(
        avg("INCOME").alias("AVG_COUNTY_INCOME")
    )
    # Join back to the original dataframe
    df = df.join(county_income_df, "COUNTY_NAME")
    
    # Step 4: Add high income flag
    df = df.with_column(
        "HIGH_INCOME_FLAG", 
        (col("INCOME") > col("AVG_COUNTY_INCOME")).astype(IntegerType())
    )
    
    # Step 5: Time-based rolling average
    df = df.with_column(
        "AVG_THIRTY_DAY_LOAN_AMOUNT",
        sql_expr("""
            AVG(LOAN_AMOUNT) OVER (
                PARTITION BY COUNTY_NAME 
                ORDER BY TIMESTAMP 
                RANGE BETWEEN INTERVAL '30 DAYS' PRECEDING AND CURRENT ROW
            )
        """)
    )

    return df

We can even see what the actual SQL execution plan in under the hood from defining all the feature logic above using Snowpark.

feature_df.explain()

## Now, we can create a [Snowflake Feature Store](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/overview).

### How does the Snowflake Feature Store work?
A feature store is used to store, retieve, and manage features within **feature views**. 

A **feature view** encapsulates a Python or SQL pipeline for transforming raw data into one or more related features. Feature views are organized by **entities**. An **entity** represents one or more ID columns that represents a unique row of feature data. 

Here, the entithy of interest is a loan: we create an entity based on the field LOAN_ID. 

In [ ]:
fs = FeatureStore(
    session=session, 
    database=session.get_current_database(), 
    name=session.get_current_schema(), 
    default_warehouse=session.get_current_warehouse(),
    creation_mode=CreationMode.CREATE_IF_NOT_EXIST)

In [ ]:
# First try to retrieve an existing entity definition
# If not, define a new one and register
try:
    # Retrieve existing entity
    loan_id_entity = fs.get_entity('LOAN_ENTITY') 
    print('Retrieved existing entity')
except:
    # Define new entity
    loan_id_entity = Entity(
        name = "LOAN_ENTITY",
        join_keys = ["LOAN_ID"],
        desc = "Features defined on a per loan level")
    # Register
    fs.register_entity(loan_id_entity)
    print("Registered new entity")

In [ ]:
fs.list_entities()

Now, we can create a [Feature View](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/feature-views) based on the feature engineering logic we defined in the function **create_mortgage_features** and applied to the data we read into a Snowpark DF.

In [ ]:
# Define and register feature view
loan_fv = FeatureView(
    name="Mortgage_Feature_View",
    entities=[loan_id_entity],
    feature_df=create_mortgage_features(df).select(
        ["LOAN_ID", "TIMESTAMP", "MONTH", "DAY_OF_YEAR", "DOTW", 
         "LOAN_AMOUNT", "LOAN_PURPOSE_NAME", "INCOME", "INCOME_LOAN_RATIO", 
         "AVG_COUNTY_INCOME", "HIGH_INCOME_FLAG", 
         "AVG_THIRTY_DAY_LOAN_AMOUNT"]),
    timestamp_col="TIMESTAMP",
    refresh_freq="1 day",
    refresh_mode="INCREMENTAL")

In [ ]:
# Add feature level descriptions
loan_fv = loan_fv.attach_feature_desc(
    {
        "MONTH": "Month of loan",
        "DAY_OF_YEAR": "Day of calendar year of loan",
        "DOTW": "Day of the week of loan",
        "LOAN_AMOUNT": "Loan amount in $USD",
        "INCOME": "Household income in $USD",
        "INCOME_LOAN_RATIO": "Ratio of LOAN_AMOUNT/INCOME",
        "AVG_COUNTY_INCOME": "Average household income aggregated at county level",
        "HIGH_INCOME_FLAG": "Binary flag to indicate whether household income is higher than MEDIAN_COUNTY_INCOME",
        "AVG_THIRTY_DAY_LOAN_AMOUNT": "Rolling 30 day average of LOAN_AMOUNT"
    }
)

In [ ]:
## Register (add) the feature view to our feature store) 
loan_fv = fs.register_feature_view(loan_fv, version=VERSION_NUM, overwrite=True)

Let's take a look at the existing feature views we have in our Feature Store through the Python API.

In [ ]:
fs.list_feature_views()

You can also inspect your feature view in the Feature Store UI link generated in the next cell. 

You can also click on the `Lineage` tab in the Feature View to look at the lineage of these features. 

# Feature Retrieval 
### Retrieve a [Dataset from the feature view for model training](https://docs.snowflake.com/en/developer-guide/snowflake-ml/feature-store/modeling#generating-snowflake-datasets-for-training)

We can now generate a [Snowflake Dataset](https://docs.snowflake.com/en/developer-guide/snowflake-ml/dataset), which are immutable, file-based objects that exist within your Snowpark session. They can be converted to Snowpark Dataframes, or written to persistent Snowflake objects as needed.

First, we create a **spine dataframe** which will contain the rows and entity IDs that we want to include in our training data. In this example, we are selecting all the rows in our source data. 

In [ ]:
df.limit(5)

In [ ]:
# Create a spine dataframe to select the desired rows for training. 
# In this case we are using all the rows 
spine_df = df.select("LOAN_ID","TIMESTAMP", "MORTGAGERESPONSE") #only need the features used to fetch rest of feature view
spine_df.show()

The `generate_dataset()` API will fill this spine_df with the correct feature values from a feature view that has been registered in the feature store. 

Here, we already have the feature view in memory, but we demonstrate how to load a feature view from the feature store, for reference. 

In [ ]:
mortgage_feature_view = fs.get_feature_view(
    name="MORTGAGE_FEATURE_VIEW",
    version="ML303_V1"
)

mortgage_feature_view

In [ ]:
ds = fs.generate_dataset(
    name=f"MORTGAGE_DATASET_EXTENDED_FEATURES_{VERSION_NUM}",
    # only need the features used to fetch rest of feature view
    spine_df=df.select("LOAN_ID", "TIMESTAMP", "MORTGAGERESPONSE"), 
    features=[mortgage_feature_view],
    spine_timestamp_col="TIMESTAMP",
    spine_label_cols=["MORTGAGERESPONSE"]
)

In [ ]:
ds_sp = ds.read.to_snowpark_dataframe()
ds_sp.show(5)

# Best practice: Choose the runtime that best suits each step, reduce configuration effort and operational burden

Here, we will use [Snowflake ML distributed preprocessors such as OneHotEncoder](https://docs.snowflake.com/en/developer-guide/snowflake-ml/modeling#preprocessing) which are implementations of sklearn preprocessors and run in parallel on a Snowflake **Warehouse**. 

Alternatively, one could use OSS sklearn preprocessors directly on the **Container Runtime** node. 

In [ ]:
import snowflake.ml.modeling.preprocessing as snowml
from snowflake.snowpark.types import StringType

OHE_COLS = ds_sp.select([col.name for col in ds_sp.schema if col.datatype==StringType()]).columns
OHE_POST_COLS = [i+"_OHE" for i in OHE_COLS]

# Encode categoricals to numeric columns
snowml_ohe = snowml.OneHotEncoder(input_cols=OHE_COLS, output_cols = OHE_COLS, drop_input_cols=True)
ds_sp_ohe = snowml_ohe.fit(ds_sp).transform(ds_sp)

# Rename columns to avoid double nested quotes and white space chars
rename_dict = {}
for i in ds_sp_ohe.columns:
    if '"' in i:
        rename_dict[i] = i.replace('"','').replace(' ', '_')

ds_sp_ohe = ds_sp_ohe.rename(rename_dict)
ds_sp_ohe.columns

Now, we're ready to split the processed data into train/test sets and fill any null values.

In [ ]:
train, test = ds_sp_ohe.random_split(weights=[0.70, 0.30], seed=0)

In [ ]:
cols = list(ds_sp_ohe.columns)
cols.remove("TIMESTAMP")

train = train.fillna(0, subset=cols)
test = test.fillna(0, subset=cols)

# Distributed model training & deployment

Now we demonstrate how to execute distributed model training using Snowflake's [distributed modeling classes](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/latest/modeling_distributors).

Snowflake will set up a ray cluster on all available nodes in your compute pool (CPU or GPU) and execute the distributed training job

In [ ]:
import ray
import pprint

def _format_resources(resources):
    """Convert memory fields to GB and filter out internal node tags."""
    formatted = {}
    for k, v in resources.items():
        # Skip internal node identifiers
        if k.startswith("node:"):
            continue
        if k in {"memory", "object_store_memory"}:
            formatted[k] = f"{v / (1024 ** 3):.2f} GB"
        else:
            formatted[k] = v
    return formatted

def show_ray_cluster_resources():
    """Nicely formatted cluster-wide and node-level resource info from Ray."""
    print(" Cluster Resources:")
    cluster = _format_resources(ray.cluster_resources())
    pprint.pprint(cluster, sort_dicts=True, width=100)

    print("\n Node-Level Resources:")
    for node in ray.nodes():
        print(f"\nNode: {node['NodeManagerAddress']}")
        node_resources = _format_resources(node["Resources"])
        pprint.pprint(node_resources, sort_dicts=True, width=100)

Let's scale our resources to use all 4 nodes in our cluster using Snowflake ML's `scale_cluster` function.

In [ ]:
# Scale to max compute pool nodes
num_nodes = 4

# Suppress SIGTERM ray warnings 
logging.basicConfig(level=logging.ERROR)

# Use full path name
nb_name = "E2E_ML_NOTEBOOK_DIST"
scale_cluster(expected_cluster_size=num_nodes,
              is_async=True,
              notebook_name=nb_name)

In [ ]:
# Show number of nodes after changing cluster manager settings
show_ray_cluster_resources()

alter compute pool CP_GPU_NV_S_4 set min_nodes=4;
describe compute pool CP_GPU_NV_S_4;

 alter compute pool CP_GPU_NV_S_4 set AUTO_SUSPEND_SECS = 3600;


Now, we're ready to do multi-node, multi-GPU training.

In [ ]:
# Quiet messages from Ray
context = ray.data.DataContext().get_current() 
context.execution_options.verbose_progress = False
# context.enable_operator_progress_bars = False
# context.enable_progress_bars = False

# Best practice: Data Connectors. 
## Use Snowflake DataConnector to merge the efficiencies of Snowflake Table structures with distributed ray operation.


In [ ]:
from snowflake.ml.modeling.distributors.xgboost.xgboost_estimator import XGBEstimator, XGBScalingConfig
from snowflake.ml.data.data_connector import DataConnector

# Use Snowflake's DataConnector to efficiently connect Snowflake data to Ray
dc = DataConnector.from_dataframe(train)

# Set up the scaling configuration for multi-node and multi-GPU usage
scaling_config = XGBScalingConfig(
    num_workers=-1,            # Use all available workers
    num_cpu_per_worker=-1,     # Use all available CPU cores per worker
    use_gpu=True               # Enable GPU for training, resources_per_worker={"GPU": 1}
)

# Define XGBoost hyperparameters for GPU
hyp_params = {
    "booster": "gbtree",               # Standard boosting model
    "tree_method": "gpu_hist",         # Enables NV_GPU Histogram
    "predictor": "auto",               # Uses GPU for prediction
    "n_estimators": 100,               # Number of trees to train
    "max_depth": 5,                    # Maximum tree depth
    "grow_policy": "lossguide",        # Enables better GPU scalability by growing trees leaf-wise (instead of depth-wise)
    "max_leaves": 512,                 # Limits tree complexity; balances model capacity and memory usage when using 'lossguide'
    "learning_rate": 0.1,              # Step size for each tree
    "subsample": 0.8,                  # Fraction of samples used for each tree
    "colsample_bytree": 0.8            # Fraction of features used for each tree
}

# Define distributed xgb estimator
multi_node_gpu_xgb = XGBEstimator(
    params = hyp_params,
    scaling_config = scaling_config)

multi_node_gpu_xgb.fit(dc,
                 input_cols = train.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"]).columns,
                 label_col = "MORTGAGERESPONSE")

# Best practices: Register all models. 
## Log all your model versions for 'set and forget' experiment tracking. Use Snowpark Dataframes to track data lineage all the way back to the source table, through Feature Engineering.


### Now, we can log this distributed xgb model to Snowflake Model Registry:
- Log models with important metadata
- Manage model lifecycles
- Serve models from Snowflake runtimes

In [ ]:
#Create a snowflake model registry object 
from snowflake.ml.registry import Registry
from snowflake.ml._internal.utils import identifier
from snowflake.ml.model import model_signature

db = identifier._get_unescaped_name(session.get_current_database())
schema = identifier._get_unescaped_name(session.get_current_schema())

# Define model name
model_name = f"MORTGAGE_LENDING_MLOPS_{VERSION_NUM}"

# Create a registry to log the model to
model_registry = Registry(session=session, 
                          database_name=db, 
                          schema_name=schema,
                          options={"enable_monitoring": True})

DROP MODEL IF EXISTS {{model_name}};

# Best Practice: Set your runtime for inference with a single parameter. 
## Use SPCS Model Serving for GPU-acceleration or low-latency requirements.

In [ ]:
logging.getLogger().setLevel(logging.WARN)

# Extract xgb booster object from Snowflake optimized XGB model
gpu_booster = multi_node_gpu_xgb.get_booster()

# Log the distributed model to the model registry
dist_version_name = f'XGB_MULTI_NODE_GPU_DIST'

try:
    mv_base = model_registry.get_model(model_name).version(dist_version_name)
    print("Found existing model version!")
except:
    print("Logging new model version...")
    mv_base = model_registry.log_model(
        model_name=model_name,
        model=gpu_booster,
        version_name=dist_version_name,
        sample_input_data = train.drop(["TIMESTAMP", "LOAN_ID", "MORTGAGERESPONSE"]).limit(100), #using snowpark df to maintain lineage
        comment = """Distributed ML model for predicting loan approval likelihood.""",
        options={'relax_version': True},
        target_platforms={"WAREHOUSE", "SNOWPARK_CONTAINER_SERVICES"}
    )

In [ ]:
SHOW VERSIONS IN MODEL {{model_name}}

In [ ]:
model_registry.show_models()

Since we don't need all nodes anymore, let's scale down our cluster now.

In [ ]:
# Scale down 
num_nodes = 1

# Suppress SIGTERM ray warnings 
logging.basicConfig(level=logging.ERROR)

# Use full path
nb_name = "E2E_ML_NOTEBOOK_DIST"
scale_cluster(expected_cluster_size=num_nodes,
              # is_async=True,
              notebook_name=nb_name)

We can see that 3 of the nodes are empty now, which tells us they're no longer being considered as available.

In [ ]:
# Show number of nodes after changing cluster manager settings
show_ray_cluster_resources()

# Best practice: Model Serving via Snowflake Model Registry. 
## Logging models in the model registry simplifies model serving actions and makes scaling an afterthought

## Model Deployment to Snowpark Container Services (SPCS)

Here, we also show how to deploy a model to Snowpark Container Services as a long running service using [Model Serving](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/container).

This is useful to run GPU based inference, run very large models that do not fit in the Warehouse memory, or when you have additional package dependencies that are not met in the Warehouse. You can also create REST API endpoints for running inference using external HTTP requests. 

In [ ]:
# Define the variables we'll use to create the deployment service.
image_repo_name = "my_inference_images"
cp_name = "CP_GPU_NV_S_4"
num_spcs_nodes = '2'
service_name = 'MORTGAGE_LENDING_PREDICTION_SERVICE'

current_database = session.get_current_database().replace('"', '')
current_schema = session.get_current_schema().replace('"', '')
extended_image_repo_name = f"{current_database}.DEFAULT_SCHEMA.{image_repo_name}"
extended_service_name = f'{current_database}.DEFAULT_SCHEMA.{service_name}'

DROP SERVICE IF EXISTS {{service_name}};

Note, we're creating a service based on the model we just logged to the Model Registry: `mv_base`

You can also specify pip_requirements if your model has pip dependencies. Here we are selecting `ingress_enabled = True` to also create a REST API endpoint for our model.

In [ ]:
mv_base.create_service(
    service_name=extended_service_name,
    service_compute_pool=cp_name,
    image_repo=extended_image_repo_name,
    ingress_enabled=True,
    max_instances=int(num_spcs_nodes),
    build_external_access_integration="ALLOW_ALL_INTEGRATION"
)

In [ ]:
-- Check that the service is created
SHOW SERVICES;

In [ ]:
mv_base = model_registry.get_model(f"MORTGAGE_LENDING_MLOPS_{VERSION_NUM}").version("XGB_MULTI_NODE_GPU_DIST")
mv_base.list_services()

Now, we can run predictions on test data using this deployed SPCS Model Service.

In [ ]:
mv_base.run(test, 
            function_name = "PREDICT", 
            service_name = "DEFAULT_SCHEMA.MORTGAGE_LENDING_PREDICTION_SERVICE")

# Model Observability (Model Monitoring)
[ML Observability](https://docs.snowflake.com/en/developer-guide/snowflake-ml/model-registry/model-observability) allows you to track the quality of production models you have deployed via the Snowflake Model Registry across multiple dimensions, such as performance, drift, and volume.

Let's first save our train and test data as Snowflake tables and create a new stage for the next few steps.

In [ ]:
train.write.save_as_table(f"DEMO_MORTGAGE_LENDING_TRAIN_{VERSION_NUM}", mode="overwrite")
test.write.save_as_table(f"DEMO_MORTGAGE_LENDING_TEST_{VERSION_NUM}", mode="overwrite")

In [ ]:
session.sql("CREATE STAGE IF NOT EXISTS ML_STAGE").collect()

We can create stored procedures for running model inference. These procedures can be scheduled or orchestrated to run continuous inference.

In [ ]:
from snowflake import snowpark
from snowflake.ml.registry import Registry
import joblib
import os
import logging
from snowflake.ml.modeling.pipeline import Pipeline
import snowflake.ml.modeling.preprocessing as pp
from snowflake.snowpark.types import StringType, IntegerType
import snowflake.snowpark.functions as F

def inference_sproc(session: snowpark.Session, table_name: str, modelname: str, modelversion: str) -> str:
    database=session.get_current_database()
    schema=session.get_current_schema()
    reg = Registry(session=session)
    m = reg.get_model(model_name)  # Fetch the model using the registry
    mv = m.version(modelversion)
    
    input_table_name=table_name
    pred_col = f'{modelversion}_PREDICTION'

    # Read the input table to a dataframe
    df = session.table(input_table_name)
    # 'results' is the output DataFrame with predictions
    results = mv.run(df, 
                     function_name="predict",
                     service_name="DEFAULT_SCHEMA.MORTGAGE_LENDING_PREDICTION_SERVICE").select("LOAN_ID",'"output_feature_0"').withColumnRenamed('"output_feature_0"', pred_col)

    final = df.join(results, on="LOAN_ID", how="full")
    final = final.with_column(pred_col,
                              F.round(col(pred_col)))
    
    # Write results back to Snowflake table
    final.write.save_as_table(table_name, mode='overwrite', enable_schema_evolution=True)

    return "Success"

# Register the stored procedure
session.sproc.register(
    func=inference_sproc,
    name="model_inference_sproc",
    replace=True,
    is_permanent=True,
    stage_location="@ML_STAGE",
    packages=['joblib', 'snowflake-snowpark-python', 'snowflake-ml-python'],
    return_type=StringType()
)

## Inference Execution via stored procedure
We'll call our stored procedure 2 times which will execute inference using the deployed model on the train and test data.

In [ ]:
CALL model_inference_sproc(
    'DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}',
    '{{model_name}}', 
    '{{dist_version_name}}');

In [ ]:
CALL model_inference_sproc(
    'DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}',
    '{{model_name}}', 
    '{{dist_version_name}}');

In [ ]:
SELECT * FROM DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}} LIMIT 10

# Best practice: Use ML Observability to monitor models in in production without making copies of data or maintaining separate infrastructure.


# Create Model Monitor

We'll now create a model monitor for the model and compute the prediction drift.



In [ ]:
CREATE OR REPLACE MODEL MONITOR MORTGAGE_LENDING_DIST_MODEL_MONITOR
WITH
    MODEL={{model_name}}
    VERSION={{dist_version_name}}
    FUNCTION=predict
    SOURCE=DEMO_MORTGAGE_LENDING_TEST_{{VERSION_NUM}}
    BASELINE=DEMO_MORTGAGE_LENDING_TRAIN_{{VERSION_NUM}}
    TIMESTAMP_COLUMN='TIMESTAMP'
    PREDICTION_SCORE_COLUMNS=('XGB_MULTI_NODE_GPU_DIST_PREDICTION')  
    ACTUAL_SCORE_COLUMNS=('MORTGAGERESPONSE')
    ID_COLUMNS=('LOAN_ID')
    WAREHOUSE={{session.get_current_warehouse()}}
    REFRESH_INTERVAL='1 min'
    AGGREGATION_WINDOW='1 day';

In [ ]:
SELECT * FROM TABLE(MODEL_MONITOR_DRIFT_METRIC(
    'MORTGAGE_LENDING_DIST_MODEL_MONITOR', 'DIFFERENCE_OF_MEANS', 
    'XGB_MULTI_NODE_GPU_DIST_PREDICTION', 
    '1 DAY', 
    TO_TIMESTAMP_TZ('2024-06-01'), 
    TO_TIMESTAMP_TZ('2024-09-01')))

Since we created a REST API above, this service will run continuously. It is a good idea to drop or suspend the service if you do not need it. Compute pool will automatically suspend if no service is running.

-- DROP SERVICE IF EXISTS {{service_name}};
-- SHOW SERVICES;

-- DROP MODEL IF EXISTS {{model_name}};
-- SHOW MODELS;